In [ ]:
import sys, time, os, asyncio, glob

from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib widget
import pickle as pkl
import pandas as pd
import astropy.io.fits as pf
from astropy.time import Time, TimeDelta

from lsst_efd_client import EfdClient


In [ ]:
client = EfdClient('summit_efd')

In [ ]:
# Times to start looking at error codes
start = Time("2022-06-15 00:00:00Z", scale='utc')
end = Time("2022-07-13 12:00:00Z", scale='utc')

In [ ]:
cryoHead = await client.select_time_series('lsst.sal.ATCamera.vacuum', \
                                                ['tempCryoHead'],  start, end)
extTemp = await client.select_time_series('lsst.sal.WeatherStation.airTemperature', \
                                                ['avg1M'],  start, end)
auxtelTemp = await client.select_time_series('lsst.sal.ESS.temperature', \
                                                ['temperature0'],  start, end)

In [ ]:
print(len(cryoHead), len(extTemp), len(auxtelTemp))

In [ ]:
auxTemp = auxtelTemp['temperature0'].rolling(100).mean()

In [ ]:
plt.clf()
plt.figure(figsize=(8,11))
plt.subplots_adjust(hspace=0.5)
plt.subplot(3,1,1)
plt.title("Exterior Temperature")
extTemp['avg1M'].plot(color='blue')
plt.subplot(3,1,2)
plt.title("AuxTel Dome Temperature")
auxTemp.plot(color='green')
plt.plot([start.to_datetime(),end.to_datetime()], [9.5,9.5], color='black', ls='--')
crossing1 = datetime(2022, 6, 16, 2, 13)
plt.plot([crossing1,crossing1], [4.0,12.0], color='orange')
crossing2 = datetime(2022, 6, 21, 1, 50)
charge = datetime(2022, 6, 23, 16, 0)
plt.plot([crossing2,crossing2], [4.0,12.0], color='orange')
plt.subplot(3,1,3)
plt.title("AuxTel CryoHead Temperature")
cryoHead['tempCryoHead'].plot(color='red')
plt.plot([crossing1,crossing1], [-140.0,-130.0], color='orange')
plt.plot([crossing2,crossing2], [-140.0,-130.0], color='orange')
plt.text(charge, -125.0, "Recharge")
plt.savefig("/home/craiglagegit/DATA/AuxTel_CryoHead_Excursion_13Jul22.pdf")